# VacationPy

In [128]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame

In [129]:
# Load the csv exported in Part I to a DataFrame
file='./Output_data/cities.csv'
weather_df=pd.read_csv(file)
weather_df.head()

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed
0,1586647848,AU,Hobart,-42.88,147.33,57.00,40,62,14.99
1,1586647626,GH,Esim,4.87,-2.24,80.60,50,85,5.19
2,1586647848,AO,Kuito,-12.38,16.93,58.77,59,61,3.40
3,1586647628,GL,Qaanaaq,77.48,-69.36,11.80,88,72,3.24
4,1586647848,ZM,Nchelenge,-9.35,28.73,69.10,97,63,7.34


### Humidity Heatmap

In [130]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the Lat and Lng as locations and Humidity as the weight
locations = weather_df[["Lat", "Lon"]]
weight=weather_df["Humidity"].astype(float)

# Add Heatmap layer to map
fig=gmaps.figure(center=(44.0,-6.0),zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=weight, 
                                 dissipating=False, max_intensity=np.max(weight),
                                 point_radius=4)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria

In [131]:
# Narrow down the cities to fit weather conditions and Drop any rows will null values
hotel_df=weather_df[(weather_df["Max Tem"] <75)&(weather_df["Max Tem"] > 60)&(weather_df["Wind Speed"]< 10.0)&(weather_df["Cloudiness"]==0)].dropna(how="any").reset_index(drop=True) 
hotel_df


,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed
0,1586647873,PT,Ribeira Grande,38.52,-28.70,61.66,0,73,6.62
1,1586647880,IN,Ambikāpur,23.12,83.20,72.66,0,23,1.03
2,1586647894,PY,Mayor Pablo Lagerenza,-19.93,-60.77,71.19,0,40,3.15
3,1586647902,EG,Esna,25.29,32.55,64.40,0,45,6.35
4,1586647938,IN,Tezu,27.92,96.17,70.75,0,31,3.60
5,1586647947,CL,Collipulli,-37.95,-72.43,64.40,0,34,8.05
6,1586647967,PY,Benjamín Aceval,-24.97,-57.57,71.60,0,46,6.93


### Hotel Map

In [132]:
# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"]=""
hotel_df

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed,Hotel Name
0,1586647873,PT,Ribeira Grande,38.52,-28.70,61.66,0,73,6.62,
1,1586647880,IN,Ambikāpur,23.12,83.20,72.66,0,23,1.03,
2,1586647894,PY,Mayor Pablo Lagerenza,-19.93,-60.77,71.19,0,40,3.15,
3,1586647902,EG,Esna,25.29,32.55,64.40,0,45,6.35,
4,1586647938,IN,Tezu,27.92,96.17,70.75,0,31,3.60,
5,1586647947,CL,Collipulli,-37.95,-72.43,64.40,0,34,8.05,
6,1586647967,PY,Benjamín Aceval,-24.97,-57.57,71.60,0,46,6.93,


In [133]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Set parameters to search for hotels with 5000 meters
params = {"types": "hotel",
          "keyword": "hotels",
          "radius" : 5000,
          "key" : g_key}


for index,row in hotel_df.iterrows():
    
    lat=row['Lat']
    lon=row['Lon']
    params["location"]=f"{lat},{lon}"
    
    #Hit the Google Places API for each city's coordinates
    try: 
        response = requests.get(base_url, params=params).json()
        hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    except Exception: 
        None

In [134]:
hotel_df=hotel_df[hotel_df["Hotel Name"]!=""].reset_index(drop=True)
hotel_df

,Date,Country,City,Lat,Lon,Max Tem,Cloudiness,Humidity,Wind Speed,Hotel Name
0,1586647873,PT,Ribeira Grande,38.52,-28.70,61.66,0,73,6.62,Quinta Das Buganvílias
1,1586647880,IN,Ambikāpur,23.12,83.20,72.66,0,23,1.03,Hotel Grand Basant
2,1586647902,EG,Esna,25.29,32.55,64.40,0,45,6.35,Dahabeya Queen Farida Sailing Boat - Esna/Aswa...
3,1586647938,IN,Tezu,27.92,96.17,70.75,0,31,3.60,Hotel Taboka
4,1586647947,CL,Collipulli,-37.95,-72.43,64.40,0,34,8.05,Hotel Savoy
5,1586647967,PY,Benjamín Aceval,-24.97,-57.57,71.60,0,46,6.93,Cabaña La Celia


In [135]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lon"]]

In [136]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))